In [1]:
import requests

In [2]:
def getrecords(base_url, endpoint, headers=None, no_of_records=100, limit=30, offset=0):
  """Fetch the no_of_records from the API. Returns the fetched data in list

  Args:
      base_url (str): Base url of the API
      endpoint (str): Specific Endpoint of the API
      headers (dict, optional): headers of the API. Defaults to None.
      no_of_records (int, optional): number of records to fetch. Defaults to 100.

  Returns:
      final_data (list): Appends the data to list and returns it
  """

  final_data = []
  offset = 0
  while no_of_records!=0:
    if no_of_records<limit:
      response = requests.get(base_url+endpoint+f'?offset={offset}&limit={no_of_records}', headers=headers)
      response_data = response.json()
      final_data.extend(response_data['data'])
      return final_data

    response = requests.get(base_url+endpoint+f'?offset={offset}&limit={limit}', headers=headers)
    response_data = response.json()
    next_endpoint = response_data['next'] # /mentorskool/v1/sales?offset=101&limit=100
    params = next_endpoint.split("?")[1]  # offset=101&limit=100
    offset = params.split('&')[0].split('=')[1] # 101
    final_data.extend(response_data['data']) # len(final_data) = 100
    no_of_records = no_of_records - limit # 500 - 100 = 400

  return final_data

In [3]:
base_url = "https://zucwflxqsxrsmwseehqvjmnx2u0cdigp.lambda-url.ap-south-1.on.aws"
endpoint = "/mentorskool/v1/sales"
headers = {"access_token": "fe66583bfe5185048c66571293e0d358"}
data = getrecords(base_url=base_url,
                      endpoint=endpoint,
                      headers=headers,
                      no_of_records=500,
                      limit=100)

In [4]:
len(data)

500

In [5]:
import pandas as pd

In [6]:
def jsonToDataframe(json_inp):
    '''
    Returns the sum of two decimal numbers in binary digits.

    Parameters:
        json_inp (python dict): response of the API in form of json

    Returns:
        df (Pandas dataframe): This is the dataframe that got generated by
        parsing the json response from the API
    '''

    df = pd.json_normalize(json_inp)
    cleaned_column_names = [x if(len(x.split('.')) == 1) else x.split('.')[-1] for x in df.columns]
    df.columns = cleaned_column_names

    return df

In [7]:
sales_df = jsonToDataframe(data)

In [8]:
sales_df[sales_df.duplicated()]

,id,sales_amt,qty,discount,profit_amt,order_id,ship_mode,order_status,order_purchase_date,order_approved_at,...,product_name,colors,category,sub_category,date_added,manufacturer,sizes,upc,weight,product_photos_qty


In [9]:
sales_df.replace("null", None, inplace=True)

In [10]:
def product_size(df, product_name):
    sizes_list = df[df["product_name"]==product_name]["sizes"].unique()[0]
    if not sizes_list:
        sizes = []
        count_of_sizes = len(sizes)
    else:
        sizes = [size.strip() for size in sizes_list.split(',')]
        count_of_sizes = len(sizes)
    return count_of_sizes

In [11]:
total_sizes = []
for index in range(len(sales_df)):
    product = sales_df.iloc[index]["product_name"]
    sizes = product_size(sales_df, product)
    total_sizes.append(sizes)


sales_df["no_of_sizes"] = total_sizes
sales_df.head()

,id,sales_amt,qty,discount,profit_amt,order_id,ship_mode,order_status,order_purchase_date,order_approved_at,...,colors,category,sub_category,date_added,manufacturer,sizes,upc,weight,product_photos_qty,no_of_sizes
0,4102,2.0000,0,72.948,None,US-2017-102288,Standard Class,delivered,2017-11-24 15:58:00,2017-11-28 03:18,...,Blue,Office Supplies,Appliances,2017-01-17,None,7,826000000000,None,0,1
1,2698,261.9600,2,0.000,41.9136,CA-2014-145317,Standard Class,delivered,2018-07-11 19:46:00,2018-07-13 15:25,...,Pink,Furniture,Bookcases,2016-04-01,None,9,640000000000,None,4,1
2,6827,731.9400,3,0.000,219.582,CA-2016-118689,Standard Class,delivered,2018-05-15 20:24:00,2018-05-15 20:55,...,Pink,Furniture,Chairs,2016-11-04,None,"10,7,6,9,8",664000000000,None,2,5
3,8154,14.6200,2,0.000,6.8714,CA-2017-140151,First Class,delivered,2018-02-28 13:40:00,2018-03-01 13:35,...,Pink,Office Supplies,Labels,2016-08-01,None,11.5,887000000000,None,3,1
4,2624,957.5775,5,0.450,-383.031,CA-2017-127180,First Class,delivered,2018-01-07 21:45:00,2018-01-07 21:57,...,Blue,Furniture,Tables,2016-11-15,None,"10.5,10,8.5,8,13",888000000000,None,0,5


In [12]:
sales_df.loc[sales_df["no_of_sizes"].idxmax()]['product_name']

'Redi-Strip #10 Envelopes, 4 1/8 x 9 1/2'

In [13]:
sales_df[sales_df['product_name']=='Mitel 5320 IP Phone VoIP phone']['no_of_sizes']

8    0
Name: no_of_sizes, dtype: int64